In [11]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import re
import os
from datetime import date
#import mygene         #for alternative method of changing ensembl names to "normal" gene names
#mg = mygene.MyGeneInfo()

In [2]:
#Function for performing the PCA. Takes dataframe with expression values as input
def my_pca(df, n_pc=1, normalize=True):
    df = df.dropna(axis = 0, how = 'all') #Remove rows with only NA values. Should be none
    x = df.values.T #Set x as transpose of only the numerical values of the dataframe
    if normalize:
        x2 = preprocessing.scale(x) #Standardize the data (center to mean and scale to unit variance)
    else:
        x2 = x
    pca = PCA(n_components = n_pc) #Set PCA parameters
    pca.fit(x2) #Fit data to model
    expl = pca.explained_variance_ratio_
    #my_pca.pca = pca #Used for Patrik's dictionary method for components
    x3 = pca.fit_transform(x2) #Transform the data (apply dimensionality reduciton) and set x3 as principal components 
    out_df = pd.DataFrame(x3.transpose(), index=list(range(1,n_pc+1)), columns=df.columns) #Create dataframe with vlues from the PCA and set columnindex as the PC number 
    out_df = out_df.transpose()
    return out_df, expl

#Function for creating csv with all TFs and their target genes. For each TF, identifies urls for target gene data and fetches the info and puts it into a list that is then converted to a dataframe and given as output
def TFs_targets(kbp, chip):   
    TFrange = ('Target Genes (TSS }'+kbp+'k)')
    TF_gene_list = []
    dex = 0
    for url in chip.loc[:,TFrange]: #OBS! This takes a long time.
        try:
            TF_gene_set = pd.read_csv(url, sep='\t') #For a specific TF, read csv from url as a dataframe
            genes = TF_gene_set['Target_genes'].tolist() #Take the contents of column 'Target_genes' and puts it into a list
            TF_gene_list.append(genes) #Append the list for a specific TF to list with all TFs
            print('Genes for '+chip.index[dex]+' ('+str(dex)+') found')
        except HTTPError: #If the url can't be reached, insert 'Not found' in the list and continue (to get correct index)
            genes = ['Not found']
            TF_gene_list.append(genes) #Append message that genes were not found
            print('Genes for '+chip.index[dex]+' ('+str(dex)+') NOT found')
        dex = dex+1
    TF_gene_sets = pd.DataFrame({'Genes':TF_gene_list}, index=chip.index) #Create a dataframe from the list of TFs and their target genes
    TF_gene_sets.to_csv('../data/Transfactors/TF_gene_sets.tsv', sep='\t')
    return TF_gene_sets

#Function for replacing the ensenbl gene IDs with gene symbols/names in expression data
def ID_to_symbol(datatsv):    
    dataset = pd.read_csv(datatsv, sep='\t') 
    symbols = pd.read_csv('../data/Gene_names.txt', index_col='Gene stable ID')
    symbols = testing.drop_duplicates(keep=False)
    x = 0
    for gene in dataset.loc[:,'Gene']:
        try:
            dataset.loc[x,'Symbol'] = symbols.loc[gene,'Gene name']
            x = x + 1
        except KeyError:
            dataset.loc[x,'Symbol'] = float('NaN')
            x = x + 1
    dataset = dataset.dropna(how='any')
    dataset = dataset.set_index('Symbol')
    dataset = dataset.drop(columns='Gene')
    dataset.to_csv('../data/genes.raw.htseq2.symbols.tsv', sep='\t')
    return dataset

# The original chip_atlas_analysis_list.csv that was downloaded did not work, as it contained an extra '"' in the start and end of each row with '"' in it, and each '"' had an extra  '"'
def csv_fix(csv):
    name, ext = os.path.splitext(csv)
    newfilename = '{name}_{uid}{ext}'.format(name=name, uid='CORRECTED', ext='.csv')
    with open(csv,'r+') as csv_file:
        for line in csv_file:
            # removing starting and ending quotes of a line
            pattern1 = re.compile(r'^"|"$',re.MULTILINE)
            line = re.sub(r'^"|"$',"",line)
            # substituting escaped quote with a single quote
            pattern2 = re.compile(r'""')
            line = re.sub(r'""','"',line)
            corrected_csv = open(newfilename,'a')
            corrected_csv.write(line)
            corrected_csv.close()
    csv_df = pd.read_csv(newfilename, quotechar = '"')
    return csv_df

In [3]:
# Read csv with all all TFs and urls to files with their target genes
try:
    chip = pd.read_csv('../data/Transfactors/chip_atlas_analysis_list_CORRECTED.csv').loc[1255:1953,['Antigen','Target Genes (TSS }1k)','Target Genes (TSS }5k)', 'Target Genes (TSS }10k)']]
#    chip = pd.read_csv('../data/Transfactors/chip_atlas_analysis_list_CORRECTED.csv').loc[1255:1265,['Antigen','Target Genes (TSS }1k)','Target Genes (TSS }5k)', 'Target Genes (TSS }10k)']]
except FileNotFoundError:
    chip = csv_fix('../data/Transfactors/chip_atlas_analysis_list.csv').loc[1255:1953,['Antigen','Target Genes (TSS }1k)','Target Genes (TSS }5k)', 'Target Genes (TSS }10k)']]
#    chip = csv_fix('../data/Transfactors/chip_atlas_analysis_list.csv').loc[1255:1265,['Antigen','Target Genes (TSS }1k)','Target Genes (TSS }5k)', 'Target Genes (TSS }10k)']]
chip = chip.set_index('Antigen')

#Read csv with TFs and their target genes. If not available, fetch the data and create a csv
try: 
    TF_gene_sets = pd.read_csv('../data/Transfactors/TF_gene_sets.tsv', sep='\t', index_col='Antigen', converters={'Genes': lambda x: x.strip('[]').replace("'","").split(', ')})
except FileNotFoundError:
    TF_gene_sets = TFs_targets('10', chip)

#Read in dataset with expression values and gene symbols. If not available, replace ensembl IDs and create the csv
try:
    dataset = pd.read_csv('../data/genes.raw.htseq2.symbols.tsv', sep='\t', index_col='Symbol')
except:
    ID_to_symbol('../data/genes.raw.htseq2.tsv')

#Read in dataset with sample sepcifications
dataset2 = pd.read_csv('../data/E-MTAB-2328.sdrf.tsv', sep='\t') #Read csv with specifications of assays

Genes for Acaa2 (0) found
Genes for Acss2 (1) found
Genes for Actb (2) found
Genes for Adnp (3) found
Genes for Aebp2 (4) found
Genes for Aff3 (5) found
Genes for Aff4 (6) found
Genes for Ahcy (7) found
Genes for Ahr (8) found
Genes for Aicda (9) found
Genes for Aire (10) found
Genes for Anp32e (11) found
Genes for Ar (12) found
Genes for Arid1a (13) found
Genes for Arid3a (14) found
Genes for Arid4b (15) found
Genes for Arntl (16) found
Genes for Art1 (17) found
Genes for Ascl1 (18) found
Genes for Ascl2 (19) found
Genes for Ash1l (20) found
Genes for Ash2l (21) found
Genes for Asxl1 (22) found
Genes for Atf2 (23) found
Genes for Atf3 (24) found
Genes for Atf4 (25) found
Genes for Atf7 (26) found
Genes for Atf7ip (27) found
Genes for Atm (28) found
Genes for Atoh1 (29) found
Genes for Atrx (30) found
Genes for Auts2 (31) found
Genes for Bach1 (32) found
Genes for Bach2 (33) found
Genes for Bap1 (34) found
Genes for Batf (35) found
Genes for Batf3 (36) found
Genes for Baz1a (37) found


Genes for Lmnb1 (300) found
Genes for Lmo2 (301) found
Genes for Lmx1b (302) found
Genes for Lrrfip1 (303) found
Genes for Lyl1 (304) found
Genes for Maf (305) found
Genes for Mafa (306) found
Genes for Mafb (307) found
Genes for Maff (308) found
Genes for Mafg (309) found
Genes for Mafk (310) found
Genes for Mapk8 (311) found
Genes for Max (312) found
Genes for Maz (313) found
Genes for Mbd1 (314) found
Genes for Mbd2 (315) found
Genes for Mbd3 (316) found
Genes for Mcrs1 (317) found
Genes for Mecp2 (318) found
Genes for Med1 (319) found
Genes for Med12 (320) found
Genes for Med23 (321) found
Genes for Med24 (322) found
Genes for Med26 (323) found
Genes for Mef2a (324) found
Genes for Mef2c (325) found
Genes for Mef2d (326) found
Genes for Meis1 (327) found
Genes for Men1 (328) found
Genes for Mettl3 (329) found
Genes for Mitf (330) found
Genes for Mllt3 (331) found
Genes for Mnx1 (332) found
Genes for Morc3 (333) found
Genes for Mphosph8 (334) found
Genes for Mpnd (335) found
Genes f

Genes for Taf2 (595) found
Genes for Taf3 (596) found
Genes for Taf7l (597) found
Genes for Taf9b (598) found
Genes for Tal1 (599) found
Genes for Tbl1x (600) found
Genes for Tbp (601) found
Genes for Tbpl1 (602) found
Genes for Tbr1 (603) found
Genes for Tbx19 (604) found
Genes for Tbx20 (605) found
Genes for Tbx21 (606) found
Genes for Tbx3 (607) found
Genes for Tbx4 (608) found
Genes for Tbx5 (609) found
Genes for Tcea1 (610) found
Genes for Tcf12 (611) found
Genes for Tcf3 (612) found
Genes for Tcf4 (613) found
Genes for Tcf7 (614) found
Genes for Tcf7l1 (615) found
Genes for Tcf7l2 (616) found
Genes for Tdg (617) found
Genes for Tead1 (618) found
Genes for Tead2 (619) found
Genes for Tead4 (620) found
Genes for Terf1 (621) found
Genes for Terf2ip (622) found
Genes for Tet1 (623) found
Genes for Tet2 (624) found
Genes for Tet3 (625) found
Genes for Tfam (626) found
Genes for Tfap2a (627) found
Genes for Tfap2c (628) found
Genes for Tfap4 (629) found
Genes for Tfcp2l1 (630) found
Ge

In [4]:
#Create dataframe with multi-colunindex of organ and developmental stage

#Exctract important characteristincs of samples from dataset2
chars = pd.DataFrame()
chars['assay'] = dataset2.loc[:,'Assay Name'].str.slice(stop=6)
#chars['dev_stage'] = dataset2.loc[:,'Characteristics[developmental stage]']
chars['organ'] = dataset2.loc[:,'Characteristics[organism part]']
chars = chars.drop_duplicates()
chars['dev_stage'] = [-5.5,-5.5,-2.5,-2.5,0.5,0.5,4,4,22,22,29,29,-5.5,-5.5,-2.5,-2.5,0.5,0.5,4,4,22,22,29,29]

chars = chars.set_index('assay')

#Insert the important characteristics into dataset with expression values as column index
datasetT = dataset.T
datasetT['dev_stage'] = datasetT.index.to_series().map(chars['dev_stage'])
datasetT['organ'] = datasetT.index.to_series().map(chars['organ'])
datasetT.set_index(['organ','dev_stage'], inplace=True)
#expdata = datasetT.sort_index(level = 0).T
expdata = datasetT.T
expdata


organ        liver        brain         liver         brain        liver  \
dev_stage    -5.5   -5.5  -5.5  -5.5    -2.5    -2.5  -2.5  -2.5    0.5    
Symbol                                                                     
Gnai3         4800   1646  2510  1424    4779    3478  1694  2512   2481   
Pbsn             0      0     0     0       0       0     0     0      0   
Cdc45          984    337   163   108     549     490    63    90    420   
H19          77631  31553  4113  1862  182270  140922  1289  3352  79202   
Scml2           72     25    85    54      48      38    63   117     34   
...            ...    ...   ...   ...     ...     ...   ...   ...    ...   
Vmn1r-ps47       0      0     0     0       0       0     0     0      0   
Gm22396          0      0     0     0       0       0     0     0      0   
Gm22394          0      0     0     0       1       0     0     0      0   
Vmn1r-ps147      0      0     0     0       0       0     0     0      0   
Methig1          0      0     0     0       0       4     0     0      1   

organ               ... brain       liver       brain       liver       brain  \
dev_stage     0.5   ...  4.0   4.0   22.0  22.0  22.0  22.0  29.0  29.0  29.0   
Symbol              ...                                                         
Gnai3         1138  ...  2429  1502  2125  1974  1153  1409  1543  1470  1937   
Pbsn             0  ...     0     0     0     0     0     0     0     0     0   
Cdc45          202  ...    99   123    98   110    41    40    27    32    73   
H19          72111  ...   403   996  2219  2385    34    73   151    71    47   
Scml2           17  ...    92    63    17     5    13    17     6     2    26   
...            ...  ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
Vmn1r-ps47       0  ...     0     0     0     0     0     0     0     0     0   
Gm22396          0  ...     0     0     0     0     0     0     0     0     0   
Gm22394          0  ...     0     0     0     0     0     0     0     0     0   
Vmn1r-ps147      0  ...     0     0     0     0     0     0     0     0     0   
Methig1          0  ...     0     0     0     0     0     0     0     0     0   

organ              
dev_stage    29.0  
Symbol             
Gnai3         856  
Pbsn            0  
Cdc45          29  
H19            29  
Scml2           9  
...           ...  
Vmn1r-ps47      0  
Gm22396         0  
Gm22394         0  
Vmn1r-ps147     0  
Methig1         0  

[32394 rows x 24 columns]

In [8]:
#Performing the PCA
var_expl = []
PCA_per_TF = pd.DataFrame(index=expdata.columns)
for TF in TF_gene_sets.index: 
    genesIndex = pd.DataFrame(TF_gene_sets.loc[TF,'Genes'], columns=['Genes'])
    genesIndex.set_index('Genes', inplace=True)  
    genes = genesIndex.index.intersection(expdata.index)
    TFdata = expdata.loc[genes,:]
    [res, expl] = my_pca(TFdata)
    PCA_per_TF[TF] = res
    var_expl.append([TF,expl[0],len(genes)])

df_var_expl = pd.DataFrame(expl_var, columns=['TF','Variance explained', 'Nr of genes'])
df_var_expl = df_var_expl.set_index('TF')
PCA_per_TF.sort_index(inplace=True)
display(PCA_per_TF)
display(df_var_expl)

Acaa2      Acss2       Actb        Adnp       Aebp2  \
organ dev_stage                                                            
brain -5.5        1.747293   2.444239  -0.389841   10.456071   20.742310   
      -5.5      -16.368792 -14.695345  -3.079625  -28.001705  -40.139809   
      -2.5      -10.016745  -7.607800  -1.178894   -9.961827  -12.593929   
      -2.5       12.552613  15.927837   2.642237   40.165420   69.567399   
       0.5       -6.033572  -2.342540  -0.152437    1.484007    6.427715   
       0.5        1.859274   5.662469   0.864787   17.335766   32.924194   
       4.0        9.201828  14.634429   3.144670   38.001785   66.316622   
       4.0       -5.374515  -1.173573   0.395917    3.721429   12.249583   
       22.0       6.069004  14.315439   2.701008   28.940155   53.871243   
       22.0      20.127346  28.776729   6.244283   62.873229  106.729003   
       29.0      49.080006  58.295058  12.094346  130.570287  211.498920   
       29.0       0.959082   7.717870   2.299969   19.141129   38.024271   
liver -5.5       15.100648   2.313086  -0.748672   -2.127639  -13.675117   
      -5.5      -21.549613 -24.508499  -4.593633  -52.138410  -84.165669   
      -2.5        8.967775   1.069314  -1.627141   -8.095920  -19.348690   
      -2.5        0.737843  -5.673242  -1.843599  -18.150348  -35.360876   
       0.5       -2.672261  -7.489024  -1.513259  -22.610687  -41.413032   
       0.5      -21.536325 -23.144168  -3.949341  -49.727491  -81.001449   
       4.0       32.400169  22.049210   1.943557   32.402956   37.270053   
       4.0       -7.525773 -10.686546  -2.646446  -28.938884  -49.598211   
       22.0     -11.553909 -14.899908  -1.570256  -33.156581  -58.366138   
       22.0     -15.093314 -17.615768  -2.518989  -37.153740  -64.068504   
       29.0     -18.448087 -19.830087  -2.952359  -44.373825  -73.596499   
       29.0     -22.629976 -23.539179  -3.566283  -50.655179  -82.293389   

                       Aff3       Aff4       Ahcy        Ahr      Aicda  ...  \
organ dev_stage                                                          ...   
brain -5.5        17.259539   4.404573   0.868003   7.866191   3.476843  ...   
      -5.5       -39.943489 -27.362309  -5.275965 -14.826787  -5.149560  ...   
      -2.5       -15.928864 -16.179566  -2.451338  -4.041380  -1.128816  ...   
      -2.5        60.463024  24.818623   8.083802  26.121284  10.920752  ...   
       0.5        -0.192320  -8.946137  -0.338615   2.784527   1.992556  ...   
       0.5        25.606613   5.728136   3.547043  12.592226   5.611799  ...   
       4.0        51.686346  18.700779   6.578579  24.481821   9.705400  ...   
       4.0         2.768042  -7.160156   1.874227   3.553837   1.710126  ...   
       22.0       39.569089  12.851784   6.314423  19.262161   7.731381  ...   
       22.0       88.416258  37.707608  12.568846  39.267023  15.043470  ...   
       29.0      186.404062  88.636658  26.104655  78.922509  30.888010  ...   
       29.0       24.931200   4.486510   4.459467  13.684839   5.209318  ...   
liver -5.5         0.106447  19.121834  -1.927080  -2.281798  -6.515634  ...   
      -5.5       -76.702732 -39.272114  -9.995879 -30.269061 -14.245539  ...   
      -2.5        -7.468360  11.569513  -2.152834  -5.809596  -5.114745  ...   
      -2.5       -24.701879  -1.714771  -4.107312 -12.066858  -7.666245  ...   
       0.5       -31.210093  -7.643147  -4.042826 -14.557463  -7.064359  ...   
       0.5       -73.327609 -38.577362  -9.073917 -29.578726 -12.120662  ...   
       4.0        53.142775  53.031456   4.773616  14.951110   6.198115  ...   
       4.0       -40.621999 -14.632506  -5.636756 -18.211382  -7.788758  ...   
       22.0      -47.333130 -20.950161  -5.984638 -23.815726  -5.898569  ...   
       22.0      -54.563706 -26.583157  -6.793985 -25.778460  -6.930556  ...   
       29.0      -64.331169 -32.502937  -8.265816 -29.031240  -8.663286  ...   
       29.0      -74.028045 -39.533151  -9.1

,Variance explained,Nr of genes
TF,,
Acaa2,0.486466,613
Acss2,0.483593,781
Actb,0.466215,32
Adnp,0.403340,4361
Aebp2,0.445511,10761
...,...,...
Zic3,0.419930,4240
Zkscan1,0.409957,5528
Zmiz1,0.422100,11167


In [9]:
df_var_expl.nlargest(10, ['Variance explained'])

,Variance explained,Nr of genes
TF,,
Thoc5,0.974005,3
Gcm1,0.676908,8
Sp110,0.673078,10
Art1,0.627226,8
Nlrp3,0.608480,5
Snai1,0.571641,604
Phf20,0.526773,12
Nsl1,0.513757,2451
Elk1,0.503574,4674


In [12]:
try:
    PCA_per_TF.to_csv('../exp/'+str(date.today())+'/PCA_results.csv')
    df_var_expl.to_csv('../exp/'+str(date.today())+'/Variance_explained.csv')
except FileNotFoundError:
    os.mkdir('../exp/'+str(date.today()))
    PCA_per_TF.to_csv('../exp/'+str(date.today())+'/PCA_results.csv')
    df_var_expl.to_csv('../exp/'+str(date.today())+'/Variance_explained.csv')

In [109]:
#Read csv with raw data with ensembl IDs replaced with gene symbols. If not available, create such a csv using mygene package
#try:
#    datasetsym = pd.read_csv('../data/genes.raw.htseq2.symbols.tsv', sep='\t', index_col='Gene')
#except FileNotFoundError:
#    dataset = pd.read_csv('../data/genes.raw.htseq2.tsv', sep='\t', index_col='Gene') 
#    datasetsym = dataset[:]
#    genes = datasetsym.index
#    genesyms = mg.querymany(genes, scopes='ensembl.gene', fields='symbol', as_dataframe=True)
#    genesyms = genesyms.dropna(how='any')
#    genesyms = genesyms.drop_duplicates(subset='Gene')
#    datasetsym.loc[:,'Gene'] = genesyms.loc[:,'symbol']
#    datasetsym = datasetsym.set_index('Gene')
#    datasetsym.to_csv('../data/genes.raw.htseq2.symbols.tsv', sep='\t')

#Attempt at finding gene symbols for each gene and set it as index.
#dataset = pd.read_csv('../data/genes.raw.htseq2.tsv', sep='\t', index_col='Gene') 
#datasetsym = dataset[:]
#genes = datasetsym.index
#genesyms = mg.querymany(genes, scopes='ensembl.gene', fields='symbol', as_dataframe=True)
#genesyms = genesyms.dropna(subset=['symbol'], how='any')
#print(genesyms.shape)

#print(genesyms.duplicated())
#genesyms = genesyms.drop_duplicates(subset='Gene', keep=False)
#datasetsym.loc[:,'Gene'] = genesyms.loc[:,'symbol']
#datasetsym = datasetsym.dropna(how='any')
#datasetsym = datasetsym.set_index('Gene')

#datasetsym.loc[:,'Gene'] = genesyms.loc[:,'symbol']
#datasetsym = datasetsym.dropna(how='any')
#datasetsym = datasetsym.set_index('Gene')
#datasetsym.to_csv('../data/genes.raw.htseq2.symbols.tsv', sep='\t')
